# Import Python modules & Initialize Earth Engine
#### Input from *fitoprincipe* @ github
https://github.com/baumi-berlin/ee-jupyter-contrib/blob/master/examples/getting-started/display-interactive-map.ipynb

In [13]:
import ee  # the Earth Engine Python API library
ee.Initialize()
import ipyleaflet  # an interactive mapping "widget" for the notebook
from baumiTools import baumiEE, baumiVT
import json

# Load a polygon shapefile for the extent (does not work yet)
here, we load a shapefile for the extent that we want to process only. This reduces computional time, and also will directly zoom to the location we are working. Test-case is a tile in the Chaco

In [8]:
shp = baumiVT.CopyToMem("C:/Miniconda3/envs/py37/Lib/site-packages/baumiTools/geeTools/Chiqui_Outline.shp")
lyr = shp.GetLayer()
featList = []
feat = lyr.GetNextFeature()
geom = feat.GetGeometryRef()
geom_json = json.loads(geom.ExportToJson())
geom_coord = geom_json['coordinates']
geom_EE = ee.Geometry.Polygon(coords=geom_coord)
lyr.ResetReading()

# Define the composite/metric we want to look at

In [26]:
median = baumiEE.Calculate_Seasonal_Median(year=2019, startMonth=1, endMonth=3)#, roi=geom_EE)# additional arguments: roi=feat_EE or path=227, row=28
#for win in [[4,6], [7,9],[10,12]]:
#    median = median.addBands(baumiEE.Calculate_Seasonal_Median(year=2018, startMonth=win[0], endMonth=win[1]))
bandNames = median.bandNames()
bandNames.getInfo()

['B_2019-01-03',
 'G_2019-01-03',
 'R_2019-01-03',
 'NIR_2019-01-03',
 'SWIR1_2019-01-03',
 'SWIR2_2019-01-03']

# Display a Dynamic Map
To display an Earth Engine generated tiles on the interactive map, we will use ipyleaflet's TileLayer object. First we start by defining a function that can generate a tile layer URL from an Earth Engine image object.

In [16]:
def GetTileLayerUrl(ee_image_object):
  map_id = ee.Image(ee_image_object).getMapId()
  tile_url_template = "https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}"
  return tile_url_template.format(**map_id)

In [27]:
map1 = ipyleaflet.Map(center=(-14.438411, -63.743799), zoom=4,
                      layout={'height':'600px'},
                      scroll_wheel_zoom=True)
map1.add_layer(ipyleaflet.TileLayer(url=GetTileLayerUrl(
    median.visualize(min=0, max=8000, gamma=1.5,
                     bands= ['NIR_2019-01-03', 'SWIR1_2019-01-03', 'R_2019-01-03']))))
map1

Map(center=[-14.438411, -63.743799], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title…